<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=7e4cb60bcfc4cee97ec915847b8e0b93f96656bbd99f7303dab2dc52f9671de0
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-05 16:07:35
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.41 C
-------------------
Today PnL: -2.71 L (-1.88%)
Current PnL: -27.00 L (-17.44%)
CY Booked + Current PnL: -12.71 L (-8.21%)
-------------------
Total profit:  1.45 L
Total loss:  -28.45 L
-------------------
Total Booked + Current PnL: 14.27 L (11.16%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.12%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 91.63 L (64.86%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.67%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.11,-16.62,20.04,0.09,16840.0,-16747.0,84030.0,95.09,38.0,M-SC,2.76,253.0,-0.99,0.62,10.40,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-3.90,-12.23,13.96,0.02,21387.0,-21357.0,153202.0,133.82,54.0,M-SC,15.38,234.0,-1.00,1.12,33.26,OX40N,NTT,PAINTS
43,ITC,452.00,0.47,-1.06,11.62,10.44,23010.0,-2117.0,198021.0,-40.74,49.0,X-LC,1.56,5.0,-0.09,1.45,4.45,X40,NTT,FMCG
50,MASFIN,398.61,0.17,-1.84,24.33,22.05,23401.0,-1800.0,96180.0,-15.54,60.0,H-SC,7.02,164.0,-0.08,0.71,39.80,XR,ATH,FINANCE
35,ICICIGI,2252.93,0.10,5.33,14.11,20.19,25909.0,9299.0,183619.0,-17.43,46.0,X-MC,6.60,68.0,0.36,1.35,21.26,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.00,5.27,-44.14,211.76,74.15,193532.0,-72213.0,91392.0,-264.69,44.0,H-SC,19.98,149.0,-0.37,0.67,17.15,XR,NTT,CHEMICALS
40,INDUSINDBK,1800.00,2.74,-35.76,106.87,32.89,53003.0,-27610.0,49596.0,-712.22,66.0,L-MC,6.24,259.0,-0.52,0.36,36.58,XR,NTT,BANKS
32,HCLTECH,1922.01,2.59,9.22,14.20,24.73,37521.0,22307.0,264231.0,14.03,71.0,X-LC,5.26,7.0,0.59,1.94,25.86,X40,ATH,IT
36,ICICIPRULI,790.00,2.42,3.79,26.19,30.97,48041.0,6695.0,183433.0,-20.57,57.0,X-MC,2.28,75.0,0.14,1.35,16.82,X40,ATH,INSURANCE
8,AWL,485.00,2.12,-19.99,91.85,53.50,221748.0,-60327.0,241424.0,-61.33,36.0,X-MC,1.61,58.0,-0.27,1.77,7.23,XY24,NTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,-79.90,-79.67,513.90,24.79,263096.0,-200660.0,51196.0,-80.65,7.0,X-SC,1.58,86.0,-0.76,0.38,0.00,X40N,NTT,MISC
24,EASEMYTRIP,26.40,-6.10,-57.51,242.86,45.70,182478.0,-101678.0,75137.0,-21.95,50.0,M-SC,23.59,205.0,-0.56,0.55,8.30,XY24,NTT,TRAVEL
44,JCHAC,2282.00,-5.92,-35.44,54.88,-0.01,45281.0,-45295.0,82510.0,16791.30,7.0,M-SC,1.04,233.0,-1.00,0.61,0.00,OX40N,NTT,AC
48,KPIGREEN,730.66,-5.14,-14.81,72.51,46.95,77394.0,-18562.0,106735.0,-21.09,33.0,H-SC,3.06,154.0,-0.24,0.78,33.71,MH,ATH,POWER
33,HINDUNILVR,2922.00,-5.02,-9.03,24.95,13.67,58348.0,-23203.0,233860.0,-15.78,32.0,X-LC,3.31,14.0,-0.40,1.72,10.40,XY25,NTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.73,-0.63,108.04,106.72,132411.0,-779.0,122557.0,-50.82,38.0,H-SC,1.57,139.0,-0.01,0.9,22.6,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.71,-6.71,122.87,107.91,169235.0,-9909.0,137735.0,-24.20,31.0,M-SC,11.24,216.0,-0.06,1.01,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-3.90,-12.23,13.96,0.02,21387.0,-21357.0,153202.0,133.82,54.0,M-SC,15.38,234.0,-1.00,1.12,33.26,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.11,-16.62,20.04,0.09,16840.0,-16747.0,84030.0,95.09,38.0,M-SC,2.76,253.0,-0.99,0.62,10.40,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.96,-23.87,49.05,13.47,100699.0,-64368.0,205299.0,-68.70,31.0,H-SC,2.39,158.0,-0.64,1.51,4.39,XY24,NTT,PAINTS
66,SIS,528.00,0.05,-24.43,60.46,21.26,50731.0,-27124.0,83908.0,1979.27,47.0,H-SC,4.06,166.0,-0.53,0.62,13.37,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,1.16,6.6,61.59,72.27,111254.0,11189.0,180637.0,-8.35,73.0,M-LC,2.95,99.0,0.1,1.33,13.19,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,1.16,6.60,61.59,72.27,111254.0,11189.0,180637.0,-8.35,73.0,M-LC,2.95,99.0,0.10,1.33,13.19,XR,NTT,IT
38,INDIAMART,4810.62,-0.73,-0.63,108.04,106.72,132411.0,-779.0,122557.0,-50.82,38.0,H-SC,1.57,139.0,-0.01,0.90,22.60,AR,ATH,MISC
37,IEX,219.00,-2.40,-1.38,50.71,48.63,99993.0,-2768.0,197186.0,-34.34,57.0,H-SC,14.76,136.0,-0.03,1.45,10.01,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-0.37,-1.78,38.08,35.63,78388.0,-3724.0,205850.0,-15.41,40.0,H-MC,3.42,119.0,-0.05,1.51,14.60,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.71,-6.71,122.87,107.91,169235.0,-9909.0,137735.0,-24.20,31.0,M-SC,11.24,216.0,-0.06,1.01,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,-79.90,-79.67,513.90,24.79,263096.0,-200660.0,51196.0,-80.65,7.0,X-SC,1.58,86.0,-0.76,0.38,0.00,X40N,NTT,MISC
9,BAJAJHFL,181.50,-1.46,-21.85,89.95,48.45,140953.0,-43804.0,156702.0,-27.39,12.0,X-MC,9.45,64.0,-0.31,1.15,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.07,-37.63,116.76,35.20,93708.0,-48413.0,80257.0,5.27,24.0,X-SC,14.40,92.0,-0.52,0.59,0.23,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.92,-11.70,37.70,21.58,46650.0,-16398.0,123741.0,-55.48,25.0,X-MC,5.74,56.0,-0.35,0.91,9.55,X40N,ATH,IT
54,PGHH,17907.65,-0.41,-5.98,42.27,33.76,79808.0,-12015.0,188805.0,-33.72,28.0,X-MC,4.69,57.0,-0.15,1.38,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-2.28,-24.43,117.22,64.15,210785.0,-58131.0,179820.0,-55.88,38.0,X-SC,1.49,82.0,-0.28,1.32,0.87,XY24,BTT,CEMENT
43,ITC,452.0,0.47,-1.06,11.62,10.44,23010.0,-2117.0,198021.0,-40.74,49.0,X-LC,1.56,5.0,-0.09,1.45,4.45,X40,NTT,FMCG
15,CAMS,4762.0,-79.90,-79.67,513.90,24.79,263096.0,-200660.0,51196.0,-80.65,7.0,X-SC,1.58,86.0,-0.76,0.38,0.00,X40N,NTT,MISC
8,AWL,485.0,2.12,-19.99,91.85,53.50,221748.0,-60327.0,241424.0,-61.33,36.0,X-MC,1.61,58.0,-0.27,1.77,7.23,XY24,NTT,FMCG
36,ICICIPRULI,790.0,2.42,3.79,26.19,30.97,48041.0,6695.0,183433.0,-20.57,57.0,X-MC,2.28,75.0,0.14,1.35,16.82,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,-79.90,-79.67,513.90,24.79,263096.0,-200660.0,51196.0,-80.65,7.0,X-SC,1.58,86.0,-0.76,0.38,0.00,X40N,NTT,MISC
9,BAJAJHFL,181.50,-1.46,-21.85,89.95,48.45,140953.0,-43804.0,156702.0,-27.39,12.0,X-MC,9.45,64.0,-0.31,1.15,0.00,X40N,ATH,FINANCE
54,PGHH,17907.65,-0.41,-5.98,42.27,33.76,79808.0,-12015.0,188805.0,-33.72,28.0,X-MC,4.69,57.0,-0.15,1.38,0.00,X40,ATH,FMCG
55,QUESS,424.00,-2.70,-32.00,110.08,42.84,48656.0,-20805.0,44201.0,-55.37,33.0,X-SC,37.37,83.0,-0.43,0.32,0.00,XY24,NTT,MISC
11,BATAINDIA,2096.00,-1.07,-37.63,116.76,35.20,93708.0,-48413.0,80257.0,5.27,24.0,X-SC,14.40,92.0,-0.52,0.59,0.23,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.0,-2.70,-32.00,110.08,42.84,48656.0,-20805.0,44201.0,-55.37,33.0,X-SC,37.37,83.0,-0.43,0.32,0.00,XY24,NTT,MISC
15,CAMS,4762.0,-79.90,-79.67,513.90,24.79,263096.0,-200660.0,51196.0,-80.65,7.0,X-SC,1.58,86.0,-0.76,0.38,0.00,X40N,NTT,MISC
59,RELAXO,1176.0,-0.41,-47.61,190.73,52.31,145043.0,-69114.0,76046.0,-44.25,34.0,X-SC,5.38,91.0,-0.48,0.56,1.79,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.0,-1.07,-37.63,116.76,35.20,93708.0,-48413.0,80257.0,5.27,24.0,X-SC,14.40,92.0,-0.52,0.59,0.23,X40,NTT,FOOTWEAR
51,MEDANTA,1486.0,-4.03,-2.85,27.72,24.08,34187.0,-3619.0,123331.0,-10.78,33.0,X-SC,6.02,89.0,-0.11,0.90,16.21,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.83,-12.01,35.57,19.28,108272.0,-41565.0,304391.0,-23.30,70.0,X-LC,6.53,1.0,-0.38,2.23,12.53,X40,ATH,IT
41,INFY,2275.00,1.16,10.02,40.76,54.87,140317.0,31363.0,344251.0,-14.21,70.0,X-LC,3.26,2.0,0.22,2.53,19.14,X40,BTT,IT
75,TMPV,600.00,-0.80,-16.21,69.68,42.18,161384.0,-44802.0,231608.0,-24.30,30.0,X-LC,3.97,3.0,-0.28,1.70,0.33,XY24,NTT,AUTO
81,VBL,671.64,0.51,-3.05,39.94,35.67,122299.0,-9634.0,306208.0,-14.52,60.0,X-LC,5.31,4.0,-0.08,2.25,10.38,X40N,ATH,FMCG
43,ITC,452.00,0.47,-1.06,11.62,10.44,23010.0,-2117.0,198021.0,-40.74,49.0,X-LC,1.56,5.0,-0.09,1.45,4.45,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,2.74,-35.76,106.87,32.89,53003.0,-27610.0,49596.0,-712.22,66.0,L-MC,6.24,259.0,-0.52,0.36,36.58,XR,NTT,BANKS
6,ASIANTILES,137.00,-2.51,-17.21,119.45,81.67,92768.0,-16147.0,77663.0,6977.78,38.0,L-SC,16.27,271.0,-0.17,0.57,50.40,XR,NTT,CERAMICS
50,MASFIN,398.61,0.17,-1.84,24.33,22.05,23401.0,-1800.0,96180.0,-15.54,60.0,H-SC,7.02,164.0,-0.08,0.71,39.80,XR,ATH,FINANCE
13,BSOFT,831.70,-0.72,-21.54,92.19,50.79,101733.0,-30302.0,110351.0,0.50,73.0,H-SC,5.21,151.0,-0.30,0.81,28.48,XR,ATH,IT
39,INDIGOPNTS,1408.00,-3.90,-12.23,13.96,0.02,21387.0,-21357.0,153202.0,133.82,54.0,M-SC,15.38,234.0,-1.00,1.12,33.26,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-3.90,-12.23,13.96,0.02,21387.0,-21357.0,153202.0,133.82,54.0,M-SC,15.38,234.0,-1.00,1.12,33.26,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.38,3.76,16.57,20.95,43285.0,9460.0,261228.0,2.74,80.0,X-LC,18.48,31.0,0.22,1.92,40.73,X40,ATH,PAINTS
13,BSOFT,831.70,-0.72,-21.54,92.19,50.79,101733.0,-30302.0,110351.0,0.50,73.0,H-SC,5.21,151.0,-0.30,0.81,28.48,XR,ATH,IT
40,INDUSINDBK,1800.00,2.74,-35.76,106.87,32.89,53003.0,-27610.0,49596.0,-712.22,66.0,L-MC,6.24,259.0,-0.52,0.36,36.58,XR,NTT,BANKS
41,INFY,2275.00,1.16,10.02,40.76,54.87,140317.0,31363.0,344251.0,-14.21,70.0,X-LC,3.26,2.0,0.22,2.53,19.14,X40,BTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.82
1,25,45.75
2,50,76.64


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.02
MC    30.18
LC    24.83
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.30
X40      22.44
X40N     10.89
XR       10.06
AR        9.40
XY25      9.37
OX40N     7.77
SR        1.02
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.55
X-MC    23.45
X-LC    21.25
M-SC    12.14
X-SC     6.93
H-MC     4.94
M-MC     1.43
L-SC     1.40
M-LC     1.33
H-LC     1.22
L-LC     1.03
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.40,-6.11,40.42
IT,12.81,-15.46,77.87
FINANCE,9.63,-18.22,68.18
ELECTRICAL,6.27,-9.66,49.83
PAINTS,6.20,-9.36,25.61
MISC,6.17,-54.75,118.14
INSURANCE,4.53,-2.61,38.77
PHARMA,4.06,-0.81,33.07
AUTO,3.44,-22.66,75.25


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3178954.0,21
XR,1291017.0,13
AR,1289668.0,10
X40,980056.0,14
X40N,965367.0,9
OX40N,734188.0,10
XY25,365272.0,6
SR,281303.0,2
MH,77394.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3633154.0,25
M-SC,1404413.0,15
X-MC,1285555.0,16
X-SC,975174.0,8
X-LC,868438.0,11
H-MC,404465.0,3
L-SC,270691.0,3
M-LC,111254.0,1
H-LC,67283.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1278179.0      6
           AR         886640.0      5
M-SC       XY24       798945.0      6
H-SC       XR         773100.0      7
X-SC       X40N       501993.0      3
X-MC       X40        448098.0      7
           XY24       381647.0      3
X-LC       X40        352405.0      5
H-SC       OX40N      336538.0      4
M-SC       OX40N      309313.0      5
X-SC       XY24       293628.0      3
H-SC       SR         281303.0      2
X-MC       X40N       269107.0      4
X-LC       XY24       232986.0      2
H-MC       AR         210896.0      2
X-LC       X40N       194267.0      2
H-MC       XY24       193569.0      1
X-MC       XY25       186703.0      2
L-SC       XR         182354.0      2
X-SC       X40        179553.0      2
M-SC       XR         171306.0      2
           AR         124849.0      2
M-LC       XR         111254.0      1
X-LC       XY25        88780.0      2
L-SC       OX40N       88337.0      1
H-SC       MH          77394.0      1
H-LC       AR          67283.0      1
L-MC       XR          53003.0      1
M-MC       XY25        50988.0      1
L-LC       XY25        38801.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
